In [15]:

import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch import nn, Tensor
import torch.optim as optimzer
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter # type: ignore
import time
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision.utils import make_grid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

CATEGORIES = ["buildings", "forest",
              "glacier", "mountain", 
              "sea", "street"]


In [16]:
def load_Intel_Image(dataDirectory : str):
    images = []
    labels = []
    for SingleCategory in CATEGORIES:
        category = os.path.join(dataDirectory, str(SingleCategory))
        
        for image in os.listdir(category):
            img_path = os.path.join(category, image)
            final_image = cv2.imread(img_path)
            
            if final_image is None:
                raise Exception("Problem with the images")
            img = cv2.resize(final_image, (64,64)) #Making the Picture 64 x 64
            images.append(img)
            labels.append(SingleCategory)
    
    return (np.array(images), np.array(labels))


def plot_confusion_matrix(cm, class_names):
    fig, ax = plt.subplots(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt="d",
                cmap="Blues", 
                xticklabels=class_names, 
                yticklabels=class_names,
                ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title("Confusion Matrix Intel Image Classification")
    plt.tight_layout()
    return fig

In [17]:
EPOCHS = 15
val_SIZE = 0.20
images, labels = load_Intel_Image("seg_train/")

X_train, X_val, Y_train, Y_val = train_test_split(
    np.array(images), np.array(labels), train_size=0.80 ,test_size= 1 - 0.80 , random_state=42
)

print(f"This is the shape of X_train: {X_train.shape}")
print(f"This is the shape of Y_train: {Y_train.shape}")
print(f"This is the shape of X_val: {X_val.shape}")
print(f"This is the shape of Y_val: {Y_val.shape}")

This is the shape of X_train: (11227, 64, 64, 3)
This is the shape of Y_train: (11227,)
This is the shape of X_val: (2807, 64, 64, 3)
This is the shape of Y_val: (2807,)


In [18]:
#1) Normalize Input Pixels
X_train = X_train.astype("float32") / 255.0
X_val = X_val.astype("float32") / 255.0


#2) We need to Transpose the Matrix because TF (M, H, W, NC) -> Pytorch (M, NC, H, W)
X_train = np.transpose(X_train, (0,3,1,2))
X_val = np.transpose(X_val, (0,3,1,2))

#3) Codificate the Y to Labels Encoder
Label_Encoder = LabelEncoder()
Y_train_encoded = Label_Encoder.fit_transform(Y_train)
Y_val_encoded = Label_Encoder.fit_transform(Y_val)

#4) Convert tensor of Pytorch(The numpy array of TensorFlow)
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train_encoded, dtype=torch.long)
x_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(Y_val_encoded, dtype=torch.long)

In [19]:
train_Dataset = TensorDataset(x_train, y_train)
val_DataSet = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_Dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_DataSet, batch_size=32, shuffle=True)


In [20]:
class CNN_Model(nn.Module):
    def __init__(self, num_classes: int = 6):
        super(CNN_Model, self).__init__()
        
        #First Block
        self.zero_pad = nn.ZeroPad2d(3) #Padding (3,3s)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=7, stride=2)
        self.bn1 = nn.BatchNorm2d(256)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        
        #Second Block
        self.covn2 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=6, stride=2)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.drop2 = nn.Dropout2d(0.30)
        #Third Block
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=2)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=1,padding=1)
        self.drop3 = nn.Dropout2d(0.30)
        
        
        self.conv4 = nn.Conv2d(128, 128, kernel_size=2)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool4 = nn.MaxPool2d(3, stride=1, padding=1)
        
        self.conv5 = nn.Conv2d(128, 128, kernel_size=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.pool5 = nn.MaxPool2d(3, stride=1, padding=1)
        
        self.conv6 = nn.Conv2d(128, 64, kernel_size=1)
        self.bn6 = nn.BatchNorm2d(64)
        self.pool6 = nn.MaxPool2d(3, stride=1, padding=1)
        
        
        self.fc = nn.Linear(64, num_classes)
    
    
    def forward(self, x : Tensor) -> Tensor:
        #Block 1
        x = self.zero_pad(x)
        x = nn.functional.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        #Block 2
        x = nn.functional.relu(self.bn2(self.covn2(x)))
        x = self.pool2(x)
        x = self.drop2(x)
        
        #Block 3 
        x = nn.functional.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
             
        
        #Block 4
        x = nn.functional.relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)
        
        
        #Block 5
        x = nn.functional.relu(self.bn5(self.conv5(x)))
        x = self.pool5(x)
        
        
        x = nn.functional.relu(self.bn6(self.conv6(x)))
        x = self.pool6(x)
        
        #Flatten and Final Block
        x = torch.flatten(x, 1)
        x = self.fc(x)
        #Dim = 1 == np.sum() == 1

        return x
        
        
        

In [21]:
"""convModel = torchvision.models.resnet50(weights="IMAGENET1K_V1")
convModel.fc = nn.Linear(convModel.fc.in_features, 6)
criterion = nn.CrossEntropyLoss()
optim = optimzer.AdamW(convModel.parameters(), lr=1e-4)
print(convModel.fc)"""

'convModel = torchvision.models.resnet50(weights="IMAGENET1K_V1")\nconvModel.fc = nn.Linear(convModel.fc.in_features, 6)\ncriterion = nn.CrossEntropyLoss()\noptim = optimzer.AdamW(convModel.parameters(), lr=1e-4)\nprint(convModel.fc)'

In [22]:
Cnn_model = CNN_Model()
Cnn_model.fc = nn.Linear(Cnn_model.fc.in_features, 6)
criterion = nn.CrossEntropyLoss()
optim = optimzer.AdamW(Cnn_model.parameters(), lr=1e-3, weight_decay=1e-4)


In [23]:
from torch.utils.tensorboard import SummaryWriter # type: ignore #ignore
writer = SummaryWriter(log_dir=f"runs/exp_{time.time()}")

for epoch in range(EPOCHS):
    Cnn_model.train()
    epoch_loss = 0.0
    
    for i,(batch_x, batch_y) in enumerate(train_loader):
        optim.zero_grad()
        outputs = Cnn_model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optim.step()
        
        
        if (epoch == 0) and (i == 0):
            images = batch_x[:8] #Only 8 images
            writer.add_images("Train/Images", images, global_step=epoch)
        epoch_loss += loss.item()
    
    train_loss = epoch_loss/len(train_loader)
    
    writer.add_scalar(f"Train/Loss", epoch_loss/len(train_loader), epoch)
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss/len(train_loader):.4f}")

    
    #Testing
    Cnn_model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i,(batch_x, batch_y) in enumerate(val_loader):
            outputs = Cnn_model(batch_x)
            
            loss_validation = criterion(outputs, batch_y)
            val_loss += loss_validation.item()
             
            if (epoch == 0) and (i == 0):
                images = batch_x[:8] #Only 8 images
                writer.add_images("Val/Images", images, global_step=epoch)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(batch_y.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            
    
    val_loss /= len(val_loader)
    acc = accuracy_score(all_labels, all_preds)

    writer.add_scalar("Val/Loss", val_loss, epoch)
    writer.add_scalar("Val/Accuracy", acc, epoch)

    cm = confusion_matrix(all_labels, all_preds)
    fig = plot_confusion_matrix(cm, Label_Encoder.classes_)
    writer.add_figure("Val/Confusion_Matrix", fig, epoch)
    plt.close(fig)

    print(
        f"Epoch [{epoch+1}/{EPOCHS}] "
        f"Val Loss: {val_loss:.2f} | Val Accuracy: {acc*100:.2f}%"
    )


writer.close()

Epoch [1/15], Loss: 1.0602
Epoch [1/15] Val Loss: 1.03 | Val Accuracy: 59.21%
Epoch [2/15], Loss: 0.8247
Epoch [2/15] Val Loss: 0.76 | Val Accuracy: 71.71%
Epoch [3/15], Loss: 0.7450
Epoch [3/15] Val Loss: 0.71 | Val Accuracy: 73.46%
Epoch [4/15], Loss: 0.6891
Epoch [4/15] Val Loss: 0.95 | Val Accuracy: 65.27%
Epoch [5/15], Loss: 0.6448
Epoch [5/15] Val Loss: 0.68 | Val Accuracy: 75.28%
Epoch [6/15], Loss: 0.6241
Epoch [6/15] Val Loss: 0.60 | Val Accuracy: 77.77%
Epoch [7/15], Loss: 0.5856
Epoch [7/15] Val Loss: 0.87 | Val Accuracy: 67.76%
Epoch [8/15], Loss: 0.5526
Epoch [8/15] Val Loss: 0.87 | Val Accuracy: 69.68%
Epoch [9/15], Loss: 0.5359
Epoch [9/15] Val Loss: 0.64 | Val Accuracy: 75.45%
Epoch [10/15], Loss: 0.5046
Epoch [10/15] Val Loss: 0.67 | Val Accuracy: 74.39%
Epoch [11/15], Loss: 0.4819
Epoch [11/15] Val Loss: 0.62 | Val Accuracy: 77.09%
Epoch [12/15], Loss: 0.4525
Epoch [12/15] Val Loss: 0.61 | Val Accuracy: 78.34%
Epoch [13/15], Loss: 0.4416
Epoch [13/15] Val Loss: 0.80 |

In [24]:

acc = accuracy_score(all_labels, all_preds)
writer.add_scalar("Accuracy", acc, epoch)

prec = precision_score(all_labels, all_preds, average='macro')
rec = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')


print(f"Validation Metrics:")
print(f"Accuracy:  {acc*100:.2f}%")
print(f"Precision: {prec*100:.2f}%")
print(f"Recall:    {rec*100:.2f}%")
print(f"F1-score:  {f1*100:.2f}%")


Validation Metrics:
Accuracy:  77.31%
Precision: 79.32%
Recall:    77.36%
F1-score:  77.06%


In [25]:
def evaluate_model(model, loader, criterion, name="Validation"):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_x, batch_y in loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += batch_y.size(0)
            correct += (preds == batch_y).sum().item()

    avg_loss = total_loss / len(loader)
    acc = correct / total
    print(f"Accuracy of the {name}: {acc:.4f}")
    print(f"Loss of the {name}: {avg_loss:.4f}")
    return acc, avg_loss

# Evaluate Model
train_acc, train_loss = evaluate_model(Cnn_model, train_loader, criterion, "training")
val_acc, val_loss = evaluate_model(Cnn_model, val_loader, criterion, "testing")


Accuracy of the training: 0.8838
Loss of the training: 0.3466
Accuracy of the testing: 0.7731
Loss of the testing: 0.6404
